In [23]:

import numpy as np

import scoobpsf
from scoobpsf.math_module import xp, _scipy, ensure_np_array
from scoobpsf.imshows import *
import scoobpsf.scoobi as scoobi

from purepyindi import INDIClient, SwitchState
client0 = INDIClient('localhost', 7624)
client0.start()

from purepyindi2 import IndiClient, SwitchState
client = IndiClient()
client.connect()

import time 
from importlib import reload

import lina


In [7]:
dir(IndiClient)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_property',
 '_has_connected_once',
 '_register_interest',
 'connect',
 'devices',
 'dispatch_callbacks',
 'get_properties',
 'handle_connectionstatus_change',
 'handle_message',
 'register_callback',
 'status',
 'to_json',
 'to_serializable',
 'unregister_callback']

In [7]:
tbi = scoobi.SCOOBI(dm_channel='dm00disp05')

TimeoutError: Timed out waiting for properties: {'scicam.exptime'}

In [14]:
client.wait_for_properties

AttributeError: 'IndiClient' object has no attribute 'wait_for_properties'

In [18]:
client.connect()

In [20]:
# client.wait_for_properties({'nsv571.exptime'}, timeout=10)
client['nsv571.exptime.exptime'] = 0.001

KeyError: 'No device nsv571 represented within these properties'

In [25]:
client['nsv571.exptime.exptime'] = 0.001

KeyError: 'No device nsv571 represented within these properties'

In [11]:
scoobi.move_psf(0,-5)

In [8]:
scoobi.client['fiberatten.current']

ValueError: not enough values to unpack (expected 3, got 2)